In [1]:
import torch
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import sys
sys.path.append('skip-thoughts.torch/pytorch')
from skipthoughts import UniSkip,BiSkip

dir_st = 'data/skip-thoughts'
vocab = ['robots', 'are', 'very', 'cool', '<eos>', 'BiDiBu']
biskip = BiSkip(dir_st, vocab)
uniskip = UniSkip(dir_st, vocab)

input = Variable(torch.LongTensor([
    [1,2,3,4,0], # robots are very cool 0
    [5,2,3,0,0]  # bidibu are very cool <eos>
])) # <eos> token is optional

print(input)
def generate_embeding(batch):
    top_half = uniskip(batch)
    bottom_half = biskip(batch)
    combine_skip = torch.cat([top_half,bottom_half],dim=1) 
    return combine_skip
print(generate_embeding(input))
print(generate_embeding(input).size())

D:\Users\Owen\Anaconda3\lib\site-packages\torch\nn\modules\rnn.py:46: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.25 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


tensor([[1, 2, 3, 4, 0],
        [5, 2, 3, 0, 0]])
tensor([[ 0.1793, -0.1286,  0.0964,  ..., -0.3654, -0.0070, -0.0919],
        [ 0.1372, -0.1739, -0.0526,  ..., -0.1531,  0.0139, -0.1812]],
       grad_fn=<CatBackward>)
torch.Size([2, 4800])


In [2]:
import pandas as pd
from Vocabulary import Vocabulary, preprocess

In [3]:
def make_vocab(tokens):
    voc = Vocabulary(['UNK'])
    voc.add_tokens(tokens)
    print('vocab len is {}'.format(len(voc.w2idx)))
    return voc

In [4]:
def load_data(file='story_cloze_data/cloze_test_test__spring2016 - cloze_test_ALL_test.csv'):
    '''TODO remove 10% of data for hyper param tuning'''
    df= pd.read_csv(file)
    df = df.drop('InputStoryid',axis=1)
    targets = df['AnswerRightEnding']
    df = df.drop('AnswerRightEnding',axis=1)
    df = df.drop('InputSentence1',axis=1)
    df = df.drop('InputSentence2',axis=1)
    df = df.drop('InputSentence3',axis=1)
    
    voc_str= ''
    for index, row in df.iterrows():
        voc_str+=' '.join(list(row)) + ' '
        
    df['AnswerRightEnding'] = targets
    return df, make_vocab(preprocess(voc_str))

In [ ]:
   
df, voc = load_data()


In [ ]:
df.head()

In [ ]:
df['RandomFifthSentenceQuiz1'][0],

In [ ]:
voc.get_sentence(preprocess(df['RandomFifthSentenceQuiz1'][0]))  

In [5]:
import sys
class LastSentenceDataset(Dataset):
    '''currently implements no context model. will add in last sentence later'''
    def __init__(self,file='story_cloze_data/cloze_test_test__spring2016 - cloze_test_ALL_test.csv',vocab=None):

        super().__init__()
        df, created_vocab = load_data(file)
        
        if vocab:
            self.vocab = vocab
        else:
            self.vocab = created_vocab
        #self.df = df
      
        
        self.dir_st = 'data/skip-thoughts'
        self.biskip = BiSkip(self.dir_st, self.vocab.convert_to_list())
        
        self.uniskip = UniSkip(self.dir_st, self.vocab.convert_to_list())
        
       
        self.data = self.make_data(df)
        
        
    def __getitem__(self, idx):
        """
        Args:
            idx
        Returns: skip thought embedding of ending and 0/1 if it is the right ending 

        """
        return self.data[idx]

    def __len__(self):
        """
        Returns len of the dataset
        """
        return len(self.data)
       
    def make_data(self, df):
        data = []
        total = df.index[:100]
        print('skip thought encoding dataset')
        for i in total:
            #print(row['RandomFifthSentenceQuiz1'],row['RandomFifthSentenceQuiz2'])
           
            progress(i,len(total))
            endings =  self.gen_embbeding(df.at[i,'RandomFifthSentenceQuiz1'], df.at[i,'RandomFifthSentenceQuiz2'])
            if df.at[i,'AnswerRightEnding'] == 1:
                data.append((endings[0],1))
                data.append((endings[1],0))
            else:
                data.append((endings[0],0))
                data.append((endings[1],1))
        return data
    

    def zero_pad(self,l,n):
        l = (l + n * [0])[:n]
        return l
    
    def pad_input(self,a,b):
        ed = sorted([a,b],key=len)
        longer = ed[1]
        shorter = ed[0]
        padded = self.zero_pad( shorter,len(longer))
        if shorter == a:
            return padded,b
        else: return a,padded
        
    def gen_embbeding(self,sent1,sent2):
        sent1 = preprocess(sent1)
        sent2 = preprocess(sent2)
        #remove random n token that is in one sentence
        if 'n' in sent2:
            sent2.remove('n')
        encoded_end1 = self.vocab.get_sentence(sent1)
        encoded_end2 = self.vocab.get_sentence(sent2)
        a,b = self.pad_input(encoded_end1,encoded_end2)
        
        batch = torch.LongTensor([a,b]) 
        top_half = self.uniskip(batch)
        bottom_half = self.biskip(batch)
        combine_skip = torch.cat([top_half,bottom_half],dim=1) 
        return combine_skip    
    
def progress(count, total, status=''):
    bar_len = 60
    filled_len = int(round(bar_len * count / float(total)))

    percents = round(100.0 * count / float(total), 1)
    bar = '=' * filled_len + '-' * (bar_len - filled_len)

    sys.stdout.write('[%s] %s%s ...%s\r' % (bar, percents, '%', status))
    sys.stdout.flush()

In [ ]:
#the Dataset constructor or something has a memory leak 
#so repeated execution of this cell can use lots of memory fast. be carefull
l = LastSentenceDataset()
tensor =  l.gen_embbeding('My friends decided to have more fun on time','My friends decided to keep inviting me out as I am so much fun',)
l[0]


In [ ]:
tensor =  l.gen_embbeding('My friends ate','My friends decided to keep inviting me out as I am so much fun')
tensor

In [ ]:
len(l)

In [ ]:
test_data_set = LastSentenceDataset(file='story_cloze_data/cloze_test_test__spring2016 - cloze_test_ALL_test.csv',vocab=l.vocab)

In [ ]:
class NoContextModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        
        self.input = torch.nn.Linear(4800,256)
        self.hidden= torch.nn.Linear(256,64)
        self.output = torch.nn.Linear(64,2)
    def forward(self, inputs):
        hidden = torch.nn.functional.relu(self.input(inputs))
        hidden1 = torch.nn.functional.relu(self.hidden(hidden))
        output = torch.nn.functional.relu(self.output(hidden1))
        return output

In [6]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.hd1 = torch.nn.Linear(4800, 2400)
        self.hd2 = torch.nn.Linear(2400, 1200)
        self.hd3 = torch.nn.Linear(1200, 600)
        self.output = torch.nn.Linear(600, 2)

    def forward(self, x):
        x = torch.nn.functional.relu(self.hd1(x))
        x = torch.nn.functional.relu(self.hd2(x))
        x = torch.nn.functional.relu(self.hd3(x))
        x = self.output(x)
        print('output',x)
        return x

In [7]:
import time
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
plt.style.use('seaborn-paper')
batch_size = 64
lr = 0.1
num_epochs = 10
report_every = 1


In [14]:
dataset = LastSentenceDataset()
data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True,num_workers = 0)

vocab len is 5225
skip thought encoding dataset


In [15]:
model = Net()
if torch.cuda.is_available():
    model = model.cuda()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adagrad(model.parameters(), lr=lr)

In [16]:
tick = time.time()
epoch_losses = []
for epoch_num in range(1, num_epochs + 1):
    batch_losses = []
    for i, batch in enumerate(data_loader):
        ### YOUR CODE BELOW ###
        # Zero the gradients
        print('Zero Grad')
        optimizer.zero_grad()
        # Extract the inputs and the targets
        print('Extract')
        inputs, targets = batch
        #print(inputs.shape)
        #print(inputs)
        # Transfer the inputs and the targets to GPUs, if available
        print('2cuda start')
        if torch.cuda.is_available():
            #print("transfering to Cuda")
            inputs = torch.FloatTensor(inputs.float()).cuda()
            targets = torch.LongTensor(targets).cuda()
        print('2cuda end')
        #print(inputs.size())
        # Run the model
        print('model')
        outputs = model(inputs)

        # Compute the loss
        print('loss')
        loss = criterion(outputs,targets)
        
        
        # Backpropagate the error
        print('backprop')
        loss.backward(retain_graph=True)
        # Update the parameters
        print('step')
        optimizer.step()
        
        # Append the loss
        print('append loss')
        batch_losses.append(float (loss))
        ### YOUR CODE ABOVE ###
        epoch_loss = np.mean(np.array(batch_losses))
        epoch_losses.append(epoch_loss)

        if epoch_num % report_every == 0:
            tock = time.time()
            print("Epoch {}. Loss {:.4f}. Elapsed {:.0f} seconds".format(epoch_num, epoch_loss, tock-tick))

print("Total time elapsed: {:.0f} minutes".format((tock-tick)/60))

Zero Grad
Extract
2cuda start
2cuda end
model
output tensor([[-0.0094, -0.0243],
        [-0.0161, -0.0224],
        [-0.0096, -0.0200],
        [-0.0177, -0.0156],
        [-0.0114, -0.0183],
        [-0.0188, -0.0193],
        [-0.0170, -0.0190],
        [-0.0203, -0.0254],
        [-0.0189, -0.0190],
        [-0.0136, -0.0146],
        [-0.0100, -0.0199],
        [-0.0089, -0.0162],
        [-0.0164, -0.0224],
        [-0.0146, -0.0231],
        [-0.0120, -0.0119],
        [-0.0096, -0.0286],
        [-0.0161, -0.0178],
        [-0.0178, -0.0149],
        [-0.0149, -0.0165],
        [-0.0129, -0.0270],
        [-0.0152, -0.0195],
        [-0.0199, -0.0113],
        [-0.0124, -0.0177],
        [-0.0119, -0.0266],
        [-0.0103, -0.0134],
        [-0.0157, -0.0139],
        [-0.0145, -0.0195],
        [-0.0184, -0.0114],
        [-0.0183, -0.0132],
        [-0.0137, -0.0147],
        [-0.0105, -0.0238],
        [-0.0117, -0.0138],
        [-0.0097, -0.0126],
        [-0.0223, -0.01

step
append loss
Epoch 2. Loss 24613.2559. Elapsed 276 seconds
Zero Grad
Extract
2cuda start
2cuda end
model
output tensor([[ -5317.3389,   4776.2275],
        [ -1517.7946,   1391.8757],
        [ -2350.8384,   2164.2505],
        [  -674.3380,    650.4410],
        [-17340.1035,  15512.9756],
        [ -1885.2577,   1724.0195],
        [ -1426.1622,   1331.7083],
        [ -1052.3207,    986.1097],
        [ -1139.6412,   1066.4955],
        [  -840.0510,    801.1550],
        [ -2145.5530,   1975.3799],
        [  -387.3616,    380.2690],
        [ -4203.3784,   3780.5032],
        [ -3458.4080,   3154.8250],
        [ -1481.0881,   1380.0093],
        [-15188.8389,  13587.7705],
        [  -726.4615,    695.8766],
        [ -1719.2499,   1600.1116],
        [ -1110.8185,   1034.7306],
        [  -532.9036,    521.9937],
        [ -1869.6512,   1721.4695],
        [ -8918.4834,   8032.9800],
        [ -1300.8760,   1217.5925],
        [ -2036.7399,   1882.2820],
        [  -416.4313

step
append loss
Epoch 3. Loss 377.2505. Elapsed 548 seconds
Zero Grad
Extract
2cuda start
2cuda end
model
output tensor([[   655.8593,   -632.3578],
        [   271.4765,   -269.2919],
        [   725.6417,   -713.6689],
        [   382.0853,   -371.6747],
        [   685.9072,   -675.1390],
        [   571.1135,   -562.0013],
        [   538.9349,   -530.8061],
        [  1302.3403,  -1309.5431],
        [   562.3139,   -554.3317],
        [   645.4698,   -641.0229],
        [   636.1023,   -630.4728],
        [   724.6978,   -722.5017],
        [  1060.1489,  -1051.4016],
        [   300.8879,   -297.6644],
        [   354.0792,   -352.0172],
        [   520.0673,   -516.5276],
        [   372.5850,   -360.9778],
        [   725.3463,   -701.8275],
        [   534.6083,   -534.2447],
        [   529.6607,   -530.7571],
        [   715.9833,   -694.0169],
        [   593.1832,   -589.9510],
        [   658.4429,   -646.4240],
        [   398.8453,   -393.4448],
        [   362.0823, 

step
append loss
Epoch 4. Loss 80.6807. Elapsed 902 seconds
Zero Grad
Extract
2cuda start
2cuda end
model
output tensor([[ -68.1282,   99.6032],
        [ -12.5305,   38.2364],
        [ -26.2086,   50.3032],
        [ -33.0761,   60.4523],
        [ 258.3078, -218.3846],
        [ -18.5872,   44.2326],
        [ -21.0602,   52.9724],
        [ -24.0254,   48.9163]], device='cuda:0', grad_fn=<AddmmBackward>)
loss
backprop
step
append loss
Epoch 4. Loss 69.6521. Elapsed 920 seconds
Zero Grad
Extract
2cuda start
2cuda end
model
output tensor([[ 3.3923e+01, -1.3524e+01],
        [ 4.2705e+01, -2.2587e+01],
        [ 5.0880e+01, -2.5268e+01],
        [ 1.9588e+01,  1.8307e+01],
        [ 6.6481e+01, -4.7755e+01],
        [ 4.1557e+01, -2.6533e+01],
        [ 1.1496e+02, -8.2924e+01],
        [ 7.1739e+01, -4.3840e+01],
        [ 4.8724e+01, -2.2800e+01],
        [ 1.2778e+02, -8.7848e+01],
        [ 9.3783e+01, -7.8831e+01],
        [ 1.3221e+02, -1.0042e+02],
        [ 3.8495e+01, -1.4014

step
append loss
Epoch 5. Loss 34.9212. Elapsed 1165 seconds
Zero Grad
Extract
2cuda start
2cuda end
model
output tensor([[ 2.0849e+01,  3.4889e+00],
        [ 1.9496e+01,  5.1358e+00],
        [ 1.9874e+02, -1.7173e+02],
        [ 8.7995e+01, -7.1442e+01],
        [ 1.6620e+02, -1.3854e+02],
        [ 1.5740e+01,  1.0090e+01],
        [ 3.1067e-02,  1.7550e+01],
        [ 8.4648e+01, -6.3068e+01],
        [-1.2843e+01,  5.6244e+01],
        [ 1.7878e+01,  1.0441e+01],
        [-1.2196e+04,  1.0914e+04],
        [-2.6801e+00,  1.7996e+01],
        [ 1.0787e+01,  8.6974e+00],
        [ 5.8943e-01,  2.1572e+01],
        [ 3.1951e+01, -9.7871e+00],
        [ 5.4441e+00,  1.3620e+01],
        [ 5.3699e+00,  1.7644e+01],
        [ 1.3817e+02, -1.0172e+02],
        [ 4.7393e+01, -4.7402e+00],
        [ 3.0419e+01, -1.0695e+01],
        [ 2.5744e+00,  1.8239e+01],
        [-1.4983e+01,  4.2318e+01],
        [ 1.3390e+02, -1.1948e+02],
        [ 9.1159e+00,  1.0519e+01],
        [ 2.6255e+01, 

step
append loss
Epoch 7. Loss 5.1189. Elapsed 1474 seconds
Zero Grad
Extract
2cuda start
2cuda end
model
output tensor([[ 1.9588e+01,  7.6510e-01],
        [ 2.5901e+01, -6.0887e-01],
        [-1.9061e+01,  5.9068e+01],
        [ 1.1674e+01,  5.0763e+00],
        [ 7.5674e+01, -6.1060e+01],
        [ 1.2926e+02, -9.5474e+01],
        [ 5.0726e+01, -4.0578e+01],
        [ 1.0231e+01,  1.0317e+01],
        [ 2.7308e+02, -2.4625e+02],
        [ 3.5268e+01, -2.2785e+01],
        [ 3.6914e+01, -1.3925e+01],
        [ 1.0576e+01,  4.1146e+00],
        [ 8.4451e+00,  1.1585e+01],
        [ 6.5819e+01, -3.2919e+01],
        [-6.7432e+00,  3.3424e+01],
        [-2.1366e+01,  5.1899e+01],
        [ 5.8216e+01, -3.8761e+01],
        [-1.4768e+04,  1.3240e+04],
        [ 7.5618e+01, -5.5424e+01],
        [ 1.3970e+01,  2.8700e+00],
        [ 1.6250e+01,  7.6468e-01],
        [-3.5611e+03,  3.2043e+03],
        [ 8.9108e+00,  1.0879e+01],
        [ 4.1140e+01, -1.1075e+01],
        [-6.5865e+00,  

step
append loss
Epoch 8. Loss 1.3310. Elapsed 1787 seconds
Zero Grad
Extract
2cuda start
2cuda end
model
output tensor([[ 5.0728e-01,  1.3987e+01],
        [ 1.6078e+01,  7.9092e-01],
        [ 5.3348e+01, -1.9113e+01],
        [ 3.6839e+01, -1.1314e+01],
        [ 1.4338e+02, -1.1458e+02],
        [ 7.5617e+01, -5.1877e+01],
        [-1.4179e+01,  4.5799e+01],
        [ 1.1917e+00,  1.4623e+01],
        [ 1.4585e+01,  7.0435e+00],
        [ 1.2165e+01,  5.6382e+00],
        [ 1.4767e+01,  6.6659e+00],
        [-1.4814e+00,  1.9821e+01],
        [ 5.2332e+00,  1.4644e+01],
        [ 6.0071e+01, -3.7985e+01],
        [-6.9116e-01,  1.3943e+01],
        [ 2.5679e+01, -1.2093e+01],
        [ 4.1452e+00,  1.0509e+01],
        [ 1.0065e+02, -8.8381e+01],
        [ 7.2068e+01, -4.8139e+01],
        [ 2.2149e+01, -1.2570e+01],
        [ 1.2463e+02, -9.9909e+01],
        [-5.9031e+00,  2.1082e+01],
        [-1.9500e+00,  1.2871e+01],
        [-1.4478e+01,  4.9548e+01],
        [-5.4626e+03,  

step
append loss
Epoch 9. Loss 1.6514. Elapsed 2143 seconds
Zero Grad
Extract
2cuda start
2cuda end
model
output tensor([[-3.1606e+00,  2.3353e+01],
        [ 1.0133e+01,  3.8427e+00],
        [ 2.0979e+01, -1.0877e+00],
        [-1.2088e+04,  1.0820e+04],
        [ 1.7519e+01,  1.0352e+00],
        [ 6.3004e-02,  1.6182e+01],
        [ 9.5293e+00,  9.8458e+00],
        [ 1.3034e+02, -1.0611e+02]], device='cuda:0', grad_fn=<AddmmBackward>)
loss
backprop
step
append loss
Epoch 9. Loss 1.2557. Elapsed 2154 seconds
Zero Grad
Extract
2cuda start
2cuda end
model
output tensor([[ 5.8984e+00,  8.0564e+00],
        [ 2.8687e+01, -1.5730e+01],
        [ 1.6548e+01, -6.0667e+00],
        [ 1.9308e+01, -1.8186e-01],
        [ 1.6561e+01, -6.3845e+00],
        [ 3.4421e+01, -3.3664e+00],
        [-6.9664e+00,  4.0420e+01],
        [ 8.6612e+00,  7.2022e+00],
        [ 4.6429e+01, -1.0016e+01],
        [ 5.8247e+00,  1.1637e+01],
        [ 6.8732e+01, -4.9048e+01],
        [ 2.1396e+01,  6.0669e-01

step
append loss
Epoch 10. Loss 0.8686. Elapsed 2405 seconds
Total time elapsed: 40 minutes


In [ ]:
next(model.parameters()).is_cuda

In [ ]:
targets.is_cuda